<a href="https://colab.research.google.com/github/viniciusriosfuck/datasus2012/blob/main/BrCitiesPopulationByAgeSex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objetivo
Este notebook extrai dados demográficos por cidades do Brasil da página do DataSUS e os consolida em tabelas a serem usadas em análises.

# Fonte dos dados: DataSus2012
Mais informações em:
http://tabnet.datasus.gov.br/cgi/ibge/popdescr.htm#resumo



## Baixar arquivo
ftp://ftp.datasus.gov.br/dissemin/publicos/IBGE/POP/popbr12.zip

Extrair POPBR12.csv

SITUACAO: situação de domicílio, 1 dígito, segundo a classificação:
* 1: Urbano
* 2: Rural
* 3: Não levantado

In [1]:
def dct_fx_etaria():
    """
        Build dict for fx_etaria mapping
    """
    dct1 = {str(x).zfill(2) + str(x).zfill(2):
            str(x).zfill(2) + ' anos'
            for x in range(20)}
    dct1['0000'] = 'menor de 1 ano'
    dct1['0101'] = '01 ano'

    lst_lower = [lower for lower in range(0,76,5)]
    lst_upper = [upper for upper in range(4,80,5)]
    dct_rng = {str(lower).zfill(2) + str(upper).zfill(2):
            str(lower).zfill(2) + ' a ' + str(upper).zfill(2) + ' anos'
            for lower,upper in zip(lst_lower,lst_upper)}

    dct = {**dct1, **dct_rng}
    dct['8099'] = '80 anos e mais'
    #dct['I000'] = 'idade ignorada'
    dct = {int(k):v for k,v in dct.items()}

    d_map = {int(k):v for k,v in dct1.items()}

    for k in d_map.keys():
        if k < 500:
            d_map[k] = dct_rng['0004']
        elif k < 1000:
            d_map[k] = dct_rng['0509']
        elif k < 1500:
            d_map[k] = dct_rng['1014']
        elif k < 2000:
            d_map[k] = dct_rng['1519']
        else:
            raise 'Error'

    d_fx_etaria = {**dct, **d_map}

    return d_fx_etaria

In [2]:
def replace_value_dct(df,col,dct):
    df[col] = df[col].map(dct)
    pass

In [3]:
def group_sum(df, col_val, col_group):
    df[col_val] = (
        df.groupby(col_group)
            [col_val].transform('sum')
    )
    df.drop_duplicates(inplace=True)
    df.drop(columns='MUNIC_RES', inplace=True)
    pass

In [4]:
def cod_mun_filter(df, cod_mun):
    return df.loc[df['MUNIC_RES'] == cod_mun]

In [5]:
def get_age_sex(df_cod_mun):
    return df_cod_mun.pivot(index=["FXETARIA"],
                            columns=["SEXO"],
                            values="POPULACAO")

In [6]:
def group_by_young_elderly(df_age_sex):
    d_old = {
    'young':[
        '00 a 04 anos',
        '05 a 09 anos',
        '10 a 14 anos',
        '15 a 19 anos',
        '20 a 24 anos',
        '25 a 29 anos',
        '30 a 34 anos',
        '35 a 39 anos',
        '40 a 44 anos',
        '45 a 49 anos',
        '50 a 54 anos',
        '55 a 59 anos'
    ],
    'elderly': [
        '60 a 64 anos',
        '65 a 69 anos',
        '70 a 74 anos',
        '75 a 79 anos',
        '80 anos e mais'
    ]
    }
    d_young_elderly = {k: oldk for oldk, oldv in d_old.items() for k in oldv}
    
    df_young_elderly = df_age_sex.copy()
    df_young_elderly.reset_index(inplace=True)

    replace_value_dct(df_young_elderly, col="FXETARIA", dct=d_young_elderly)
    
    df_young_elderly = (
        df_young_elderly.groupby('FXETARIA').sum()
        .sort_index(ascending=False)
    )

    return df_young_elderly 


In [7]:
def import_POPBR12():
    url = "https://github.com/viniciusriosfuck/datasus2012/blob/main/POPBR12.csv?raw=true"
    df = pd.read_csv(url)
    df.drop(columns=['SITUACAO','ANO'], inplace=True)
    return df

In [8]:
def replace_fx_etaria(df):
    d_fx_etaria = dct_fx_etaria()
    replace_value_dct(df, col="FXETARIA", dct=d_fx_etaria)
    pass

In [9]:
def replace_sex(df):
    d_sex = {1:'Masculino', 2:'Feminino'}
    replace_value_dct(df, col="SEXO", dct=d_sex)
    pass

# Código dos Municípios: IBGE

## Fonte dos dados: IBGE
Mais informações em:
https://www.ibge.gov.br/explica/codigos-dos-municipios.php

## Baixar arquivo
ftp://geoftp.ibge.gov.br/organizacao_do_territorio/estrutura_territorial/divisao_territorial/2018/DTB_2018.zip

Extrair RELATORIO_DTB_BRASIL_MUNICIPIO.xls

Código antigo do IBGE: 6 dígitos
ftp://geoftp.ibge.gov.br/organizacao_do_territorio/redes_e_fluxos_geograficos/ligacoes_aereas/bases_de_dados/xls/cidades.xls

In [10]:
def import_cidades():
    url = "https://github.com/viniciusriosfuck/datasus2012/blob/main/cidades.xls?raw=true"
    df_mun = pd.read_excel(url)
    lst_col = ["UF", "CodUF", "Codmundv", "Codmun", "nomemun"]
    df_mun = df_mun[lst_col]
    return df_mun

In [11]:
def get_mun_name(df,cod_col,name_col,cod_mun):
    mask = df[cod_col] == cod_mun
    mun_name = df.loc[mask,name_col].item()
    print(mun_name)
    return mun_name

In [12]:
def export_excel(df,mun_name):
    filename = mun_name + '.xlsx'
    df.to_excel(filename, encoding='utf-8')#, index=False)
    pass

In [13]:
def remove_digito_verificador_cod_mun(df):
    df["CodigoMunicipioSemDigitoVerificador"] = (
    df["Código Município Completo"]
    .astype(str)
    .str
    .slice(stop=-1)
    .astype(int)
    )
    pass

# Exemplo

## TODO
- [ ] change order from replace, group, filter to filter, group, replace
-- [x] order changed to filter, replace, group
- [ ] check warnings

In [14]:
import pandas as pd

df = import_POPBR12()
cod_mun = 420540 # 110001 # 

df_cod_mun = cod_mun_filter(df, cod_mun)
replace_fx_etaria(df_cod_mun)
replace_sex(df_cod_mun)
group_sum(df_cod_mun,
          col_val='POPULACAO',
          col_group=['MUNIC_RES','SEXO','FXETARIA'])
df_cod_mun.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

,SEXO,FXETARIA,POPULACAO
290202,Masculino,00 a 04 anos,11999
290207,Masculino,05 a 09 anos,12375
290212,Masculino,10 a 14 anos,14895
290217,Masculino,15 a 19 anos,16687
290222,Masculino,20 a 24 anos,21439


In [15]:
df_age_sex = get_age_sex(df_cod_mun)
df_age_sex

SEXO,Feminino,Masculino
FXETARIA,,
00 a 04 anos,11485,11999
05 a 09 anos,12277,12375
10 a 14 anos,14507,14895
15 a 19 anos,16809,16687
20 a 24 anos,20943,21439
25 a 29 anos,22488,22609
30 a 34 anos,19841,19414
35 a 39 anos,17105,16383
40 a 44 anos,16273,14653


In [16]:
df_young_elderly = group_by_young_elderly(df_age_sex)
df_young_elderly

SEXO,Feminino,Masculino
FXETARIA,,
young,195685,187680
elderly,28684,21109


In [17]:
df_young_elderly.loc['elderly'].sum() / (
    df_young_elderly.loc['young'].sum() + df_young_elderly.loc['elderly'].sum()   
)

0.11495343500524058

In [18]:
df_mun = import_cidades()
mun_name = get_mun_name(df=df_mun,
                        cod_col="Codmun",
                        name_col="nomemun",
                        cod_mun=cod_mun)
export_excel(df=df_age_sex, mun_name=mun_name)

Florianópolis


# TODO

Ver outras possíveis fontes de dados para população:
* https://censo2010.ibge.gov.br/sinopse/index.php?dados=26&uf=35
* http://cidades.ibge.gov.br/xtras/perfil.php?codmun=350040

Se input com dígito verificador

In [19]:
url = "https://github.com/viniciusriosfuck/datasus2012/blob/main/RELATORIO_DTB_BRASIL_MUNICIPIO.xls?raw=true"
df_mun_2 = pd.read_excel(url)

remove_digito_verificador_cod_mun(df_mun_2)
mun_name = get_mun_name(df=df_mun_2,
                        cod_col="CodigoMunicipioSemDigitoVerificador",
                        name_col="Nome_Município",
                        cod_mun=cod_mun)

Florianópolis
